## Check GIS specification

To achieve this task in Python, you can use libraries such as os for directory traversal, fiona for handling shapefiles (.shp), and geopandas for general geospatial data handling. Additionally, openpyxl can be used for writing the results to an Excel file. Here is a step-by-step guide:

1) Check the folder for .shp and .dgb files.
2) For each file, check the coordinate system and metadata.
3) Generate an Excel file with this information.

In [1]:
import shutil
import os
import arcpy

# === Set version info ===
old_version = "10"
new_version = "11"

# === Define paths ===
base_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD"
folder_template = "005356185-{} - THO - Master Construction Drawing"

## 1) Copies the folder from old version to a new version (e.g., 005356185-10--> 005356185- 11)


In [2]:
src_folder = os.path.join(base_path, folder_template.format(old_version))
dst_folder = os.path.join(base_path, folder_template.format(new_version))

# === Step 1: Copy the folder ===
try:
    shutil.copytree(src_folder, dst_folder)
    print(f"✅ Folder copied from:\n{src_folder}\nto:\n{dst_folder}")
except FileExistsError:
    print(f"⚠️ Destination folder already exists:\n{dst_folder}")
 
except FileNotFoundError:
    print(f"❌ Source folder not found:\n{src_folder}")
  
except Exception as e:
    print(f"❌ Error copying folder:\n{e}")


❌ Source folder not found:
\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185-10 - THO - Master Construction Drawing


## 2) Renames all files inside the copied folder, e.g. 005356185-10.. with 005356185-11...

In [3]:
# Rename files inside the copied folder ===
renamed_files = 0
for root, dirs, files in os.walk(dst_folder):
    for filename in files:
        if f"-{old_version}" in filename:
            old_path = os.path.join(root, filename)
            new_filename = filename.replace(f"-{old_version}", f"-{new_version}")
            new_path = os.path.join(root, new_filename)
            os.rename(old_path, new_path)
            renamed_files += 1
            print(f"🔁 Renamed: {filename} → {new_filename}")

print(f"✅ Finished. {renamed_files} files renamed in folder version {new_version}.")

✅ Finished. 0 files renamed in folder version 11.


## 3) MONOPILES

- 🌐 Open: [https://rwe.vissim.no/flexview](https://rwe.vissim.no/flexview) to see the latest update, click the **"ℹ️" (Info)** icon in the lower-left corner for more information.

### 📌 As Built Monopiles from JDN & AS BUILT: Monopile Outer Diameter – **8.6 m**

Please complete all steps **before running the script below**:

1. 🔽 **Download PDFs**  
   - URL: [https://share.jandenul.com/](https://share.jandenul.com/)  
   - **Username**: DKTRTI,  **Password**: RKHJEB5LvcnVZw2

2. 💾 **Save PDFs** here:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN*


3. 📋 **Extract and copy relevant data** into this Excel file:  
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.xlsx*


4. 📤 **Export the cleaned data to CSV**: 
   *\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv*


### ⚠️ CRITICAL NOTE: Before Running the Code

- ❗ **Ensure that the `005356185_THOR_Master_Construction_Drawing.aprx` project is closed** before running the script — otherwise layer will be not in the content (run over different aprx or VS code) 

    Two files are ouput:
    - *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0*
    - *DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0*
    

- ❗ After running the script, **open the attribute table** for:  
 * 2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0* **Delete any empty rows manually**



In [10]:
# === INPUTS ===
csv_file = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250505 As Built Monopiles from JDN\DK_THO_AsBuilt_Monopile_pt_UTM32N_ta_v0.csv"
x_field = "POS_ASBUILT_EAST"
y_field = "POS_ASBUILT_NORTH"
spatial_ref = arcpy.SpatialReference(32632)  # UTM Zone 32N

# === OUTPUTS ===
point_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0"
buffer_fc = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0"


# === STEP 1: Create Point Feature Class ===
if arcpy.Exists(point_fc):
    arcpy.Delete_management(point_fc)
    print(f"✅ Point feature class was deleted: {point_fc}")

arcpy.management.XYTableToPoint(
    in_table=csv_file,
    out_feature_class=point_fc,
    x_field=x_field,
    y_field=y_field,
    coordinate_system=spatial_ref
)

print(f"✅ Point feature class created: {point_fc}")

# === STEP 2: Create 4.3m Buffer ===
if arcpy.Exists(buffer_fc):
    arcpy.Delete_management(buffer_fc)
    print(f"✅ Point feature class was deleted: {buffer_fc}")

arcpy.analysis.PairwiseBuffer(
    in_features=point_fc,
    out_feature_class=buffer_fc,
    buffer_distance_or_field="4.3 Meters",
    dissolve_option="NONE"
)

print(f"✅ Buffer feature class created: {buffer_fc}")

✅ Point feature class was deleted: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0
✅ Point feature class created: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0
✅ Point feature class was deleted: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0
✅ Buffer feature class created: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_AsBuilt_Monopile_OuterDiameter8pt6m_py_UTM32N_pt_v0


### 3.3  *ANNOTATION – As Built Monopile Location*

1. 🔁 **Copy & Rename the Map**  
   - Open the .aprx project and *Duplicate* the map named:  
     *005356185-{old_version} THOR Master Construction Drawing CAD*  
   - Rename the copy to:  
     *005356185-{new_version} THOR Master Construction Drawing CAD*`
     

2. 👁️ **Check Labels**  
   - Ensure labels are **visible** for the following feature class:  
     **As Built Monopile** - DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0`
     

3. 💬 **Convert Labels to Annotation**  
   - If labels are showing correctly:  
     → Run the **"Convert Labels to Annotation"** tool  
     → Save the annotation layer as: *DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation*
     

4. ➕ **Add to New Map**  
   - Add the newly created annotation layer to the map:  
     *005356185-{new_version} THOR Master Construction Drawing CAD*

---



In [18]:
# === Load the project ===
import arcpy

# === INPUTS ===
new_version = "11"
aprx_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\_MCD\005356185_THOR_Master_Construction_Drawing.aprx"
output_gdb = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb"
layer_name = "As Built Monopile"
output_annotation_suffix = "_Annotation"
new_map_name = f"005356185-{new_version} THOR Master Construction Drawing CAD"
reference_scale = 100

aprx = arcpy.mp.ArcGISProject(aprx_path)
target_map = next((m for m in aprx.listMaps() if m.name == new_map_name), None)
if not target_map:
    raise Exception(f"❌ Map '{new_map_name}' not found.")
print(f"✅ Map '{new_map_name}' loaded.")

# === Find the layer ===
layer = next((lyr for lyr in target_map.listLayers() if lyr.name == layer_name), None)
if not layer or not layer.supports("SHOWLABELS"):
    raise Exception(f"❌ Layer '{layer_name}' not found or doesn't support labels.")

layer.showLabels = True
print(f"✅ Labels enabled for '{layer_name}'.")


arcpy.cartography.ConvertLabelsToAnnotation(
    input_map= target_map,
    conversion_scale= 100,
    output_geodatabase= output_gdb,
    anno_suffix= output_annotation_suffix,
    extent='394631.907089481 6223623.59608526 450316.163918058 6267404.33424396 PROJCS["WGS_1984_UTM_Zone_32N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",9.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    generate_unplaced="GENERATE_UNPLACED",
    require_symbol_id=None,
    feature_linked=None,
    auto_create="AUTO_CREATE",
    update_on_shape_change="SHAPE_UPDATE",
    output_group_layer="GroupAnno",
    which_layers="SINGLE_LAYER",
    single_layer="As Built Monopile",
    multiple_feature_classes="FEATURE_CLASS_PER_FEATURE_LAYER",
    merge_label_classes="NO_MERGE_LABEL_CLASS")


# After conversion
annotation_fc = f"{output_gdb}/As_Built_Monopile{output_annotation_suffix}"
new_fc = f"{output_gdb}/DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation"     

if arcpy.Exists(annotation_fc):
    print(f"✅ Annotation feature class saved at: {annotation_fc}")
    if arcpy.Exists(new_fc):
        arcpy.Delete_management(new_fc)
    arcpy.Rename_management(annotation_fc, new_fc)
    print(f"✅✅ Renamed annotation feature class to: {new_fc}")
else:
    # Print warning if not found
    print(f"⚠️ Annotation feature class NOT found at: {annotation_fc}")
    

✅ Map '005356185-11 THOR Master Construction Drawing CAD' loaded.
✅ Labels enabled for 'As Built Monopile'.
✅ Annotation feature class saved at: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb/As_Built_Monopile_Annotation
✅✅ Renamed annotation feature class to: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb/DK_THO_AsBuilt_Monopile_pt_UTM32N_pt_v0_Annotation


## 4) SST (VESSEL – As Built Footprint – MPI Adventure)

- **Step 1:** Open **EcoCod** and search for the title: *'VAO - Adventure T*'*

- **Step 2:** Review the top search results, specifically project 005938436-01.

- **Step 3:** The relevant DWG file from ecodoc should be saved to:  
  \\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\CONSTR\2025\20250604 As Built Secondary Steel Installation from Van Oord
  
  It will come with the improve once data available


## 5) Add Layer to MCDS with Custom Fields and Values

1. **Add Fields**
   - Add a new field called *LAYER* (type: String) to the feature class.
   - Add a new field called *SYMBOL* (type: String) to the feature class.

2. **Update Field Values**
   - Replace values in the *LAYER* field with corresponding `cad_layer_name` values.
   - Replace values in the *SYMBOL* field with corresponding `ArcGIS_value` values.

> This step ensures that the layer contains standardized naming for both CAD and ArcGIS symbology references.


In [23]:
import arcpy

# === Path to feature class ===
gdb_path = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb"
name_FC = "DK_THO_JB115_JackUpBarge_li_UTM32N_v0"
fc = f"{gdb_path}\\{name_FC}"

# === Add fields if missing ===
for field in ["LAYER", "SYMBOL"]:
    if field not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, field, "TEXT")
        print(f"🆕 Field '{field}' added.")
    else:
        print(f"✅ Field '{field}' already exists.")

# === Layer mapping ===
layer_map = {
    "JUB Install":               "VESSEL - As Planned JUB Mooring Plan Installation", 
    "JUB Stand Off":             "VESSEL - As Planned JUB Mooring Plan Standoff",
    "Mooring wire Installation": "VESSEL - As Planned JUB Mooring Plan Installation",
    "Mooring wire Stand Off":    "VESSEL - As Planned JUB Mooring Plan Standoff",
}

# === Symbol mapping ===
symbol_map = {
    "VESSEL - As Planned JUB Mooring Plan Installation": "As Planned JUB Mooring Plan Installation",
    "VESSEL - As Planned JUB Mooring Plan Standoff":     "As Planned JUB Mooring Plan Standoff",
    "VESSEL - As Planned JUB Midline buoy":              "As Planned JUB Midline buoy",
    "VESSEL - As Planned JUB Anchor":                    "As Planned JUB Anchor"
}

# === Update LAYER and SYMBOL fields ===
with arcpy.da.UpdateCursor(fc, ["LAYER", "SYMBOL"]) as cursor:
    for row in cursor:
        # Apply layer mapping if applicable
        row[0] = layer_map.get(row[0], row[0])
        # Copy LAYER value to SYMBOL
        row[1] = row[0]
        # Apply symbol mapping (optional override)
        row[1] = symbol_map.get(row[1], row[1])
        cursor.updateRow(row)

print(f"✅ Updated LAYER and SYMBOL fields in: {fc}")


🆕 Field 'LAYER' added.
🆕 Field 'SYMBOL' added.
✅ Updated LAYER and SYMBOL fields in: \\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\VESSELS\VESSELS.gdb\DK_THO_JB115_JackUpBarge_li_UTM32N_v0


In [22]:


arcpy.DeleteField_management(fc, ["SYMBOL"])
print("🗑️ Field 'SYMBOL' deleted.")

🗑️ Field 'SYMBOL' deleted.
